In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import scipy
import skimage

experimental_file = r"D:\DATA\xe005_e.mat"
theoretical_file = r"D:\DATA\4ranges-volume-momspe-2d-sw.dat"

n_resample = 512

r_min_theory = 0.15
r_max_theory = 0.25

r_min_exp = 0.2
r_max_exp = 0.3

t_min = -np.pi
t_max = np.pi

r_resample_theory = np.linspace(r_min_theory, r_max_theory, n_resample)
r_resample_exp = np.linspace(r_min_exp, r_max_exp, n_resample)
t_resample = np.linspace(t_min, t_max, n_resample)


In [ ]:

theoretical_data = [
    line.split()
    for line in open(theoretical_file).readlines()
    if len(line.split()) == 6
]

theoretical_data = np.array(theoretical_data, dtype=float)
pr, theta, Is = theoretical_data[:, 0], theoretical_data[:, 1], theoretical_data[:, 2:]
i1, i2, i3, i4 = Is[:, 0], Is[:, 1], Is[:, 2], Is[:, 3]

I = i2
grid_size = (len(set(pr)), len(set(theta)))
pr = pr.reshape(grid_size)
theta = theta.reshape(grid_size)

i_df = pd.DataFrame(Is, columns=['I1', 'I2', 'I3', 'I4'])
i_df['theta'] = theta.flatten()
i_df['pr'] = pr.flatten()

fig = px.histogram(i_df, x='pr', y='I1', histfunc='avg', nbins=100, opacity=1)
fig.add_trace(px.histogram(i_df, x='pr', y='I2', histfunc='avg', nbins=100, opacity=0.75).data[0])
fig.add_trace(px.histogram(i_df, x='pr', y='I3', histfunc='avg', nbins=100, opacity=0.5).data[0])
fig.add_trace(px.histogram(i_df, x='pr', y='I4', histfunc='avg', nbins=100, opacity=0.25).data[0])
fig.show()

pr = np.hstack((pr, pr))
theta_2 = theta - 2 * np.pi
theta = np.hstack((theta, theta_2))
I = I.reshape(grid_size)
I_shift = np.roll(I, int(grid_size[1] / 2), axis=1)
I += I_shift
I = np.hstack((I, I))
pr_vec = pr[:, 0]
theta_vec = theta[0, :]

rr_resample_theory, tt_resample = np.meshgrid(r_resample_theory, t_resample)
signal_th = scipy.interpolate.griddata((pr.flatten(), theta.flatten()), I.flatten(), (rr_resample_theory, tt_resample),
                                       method='linear')

px.imshow(signal_th.T, y=r_resample_theory, x=t_resample, origin='lower', aspect='auto')

In [ ]:
experimental_data = scipy.io.loadmat(experimental_file, squeeze_me=True)
yz_cut = experimental_data['yz_cut']
yv = experimental_data['yv']
zv = experimental_data['zv']
yy, zz = np.meshgrid(yv, zv)
px.imshow(yz_cut, y=yv, x=zv, origin='lower', aspect='auto').show()
rr, tt = np.sqrt(yy ** 2 + zz ** 2), -(np.arctan2(yy, zz) + np.pi / 2)

rr_resample_exp, tt_resample = np.meshgrid(r_resample_exp, t_resample)
tt_resample += np.pi / 2
tt_resample *= -1
yy_resample, zz_resample = rr_resample_exp * np.sin(tt_resample), rr_resample_exp * np.cos(tt_resample)
# Note: The experimental data is being flipped in the y-axis to match the theoretical data

signal_exp = scipy.interpolate.griddata((yy.flatten(), zz.flatten()), yz_cut.flatten(), (yy_resample, zz_resample),
                                        method='linear')
signal_exp *= rr_resample_exp

signal_exp_shift = np.roll(signal_exp, int(n_resample / 2), axis=0)
signal_exp += signal_exp_shift

px.imshow(signal_exp.T, y=r_resample_exp, x=t_resample, origin='lower', aspect='auto')

In [ ]:
from functools import partial

dog_filter = partial(skimage.filters.difference_of_gaussians, low_sigma=15, high_sigma=30)

signal_th_dog = dog_filter(signal_th)
signal_exp_dog = dog_filter(signal_exp)

upsample_factor = 16

rr_us_th, tt_us_th = np.meshgrid(r_resample_theory, np.linspace(-np.pi, np.pi, upsample_factor * n_resample))
rr_us_exp, tt_us_exp = np.meshgrid(r_resample_exp, np.linspace(-np.pi, np.pi, upsample_factor * n_resample))

signal_th_dog = scipy.interpolate.griddata(
        (rr_resample_theory.flatten(), -tt_resample.flatten()),
        signal_th_dog.flatten(),
        (rr_us_th, tt_us_th),
        method='cubic'
)

signal_exp_dog = scipy.interpolate.griddata(
        (rr_resample_exp.flatten(), -tt_resample.flatten()),
        signal_exp_dog.flatten(),
        (rr_us_exp, tt_us_exp),
        method='cubic'
)

signal_th_raw = scipy.interpolate.griddata(
        (rr_resample_theory.flatten(), -tt_resample.flatten()),
        signal_th.flatten(),
        (rr_us_th, tt_us_th),
        method='cubic'
)

r_vec_th = rr_us_th[0]
r_vec_exp = rr_us_exp[0]
t_vec = tt_us_th[:, 0]

rel_min_th = scipy.signal.argrelmin(signal_th_dog, axis=0)
rel_min_th_raw = scipy.signal.argrelmin(signal_th_raw, axis=0)
rel_min_exp = scipy.signal.argrelmin(signal_exp_dog, axis=0)

f1 = px.imshow(signal_th_dog.T, y=r_vec_th, x=t_vec, origin='lower', aspect='auto')
f1.add_trace(px.scatter(x=t_vec[rel_min_th[0]], y=r_vec_th[rel_min_th[1]]).data[0])

min_theory_curve = t_vec[rel_min_th[0]], r_vec_th[rel_min_th[1]]

f1.layout.xaxis.range = [-np.pi, np.pi]
f1.layout.yaxis.range = [r_min_theory, r_max_theory]
f1.show()

f2 = px.imshow(signal_exp_dog.T, y=r_vec_exp, x=t_vec, origin='lower', aspect='auto')
f2.add_trace(px.scatter(x=t_vec[rel_min_exp[0]], y=r_vec_exp[rel_min_exp[1]]).data[0])

min_exp_curve = t_vec[rel_min_exp[0]], r_vec_exp[rel_min_exp[1]]

f2.layout.xaxis.range = [-np.pi, np.pi]
f2.layout.yaxis.range = [r_min_exp, r_max_exp]
f2.show()

f3 = px.imshow(signal_th_raw.T, y=r_vec_th, x=t_vec, origin='lower', aspect='auto')
f3.add_trace(px.scatter(x=t_vec[rel_min_th_raw[0]], y=r_vec_th[rel_min_th_raw[1]]).data[0])

min_raw_theory_curve = t_vec[rel_min_th_raw[0]], r_vec_th[rel_min_th_raw[1]]

f3.layout.xaxis.range = [-np.pi, np.pi]
f3.layout.yaxis.range = [r_min_theory, r_max_theory]
f3.show()

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.03, min_samples=2)
exp_labels = dbscan.fit(np.array(min_exp_curve).T).labels_
theory_labels = dbscan.fit(np.array(min_theory_curve).T).labels_
raw_theory_labels = dbscan.fit(np.array(min_raw_theory_curve).T).labels_

main_curve_theory_idx = np.argmax(np.bincount(theory_labels[theory_labels != -1]))
main_curve_exp_idx = np.argmax(np.bincount(exp_labels[exp_labels != -1]))
main_curve_raw_theory_idx = np.argmax(np.bincount(raw_theory_labels[raw_theory_labels != -1]))

curves_df_theory = pd.DataFrame({'theta': min_theory_curve[0], 'r': min_theory_curve[1], 'label': theory_labels,
                                 "main_curve": theory_labels == main_curve_theory_idx})
curves_df_theory.sort_values(by=['r'], inplace=True)
curves_df_exp = pd.DataFrame({'theta': min_exp_curve[0], 'r': min_exp_curve[1], 'label': exp_labels,
                              "main_curve": exp_labels == main_curve_exp_idx})
curves_df_exp.sort_values(by=['r'], inplace=True)
curves_df_raw_theory = pd.DataFrame(
        {'theta': min_raw_theory_curve[0], 'r': min_raw_theory_curve[1], 'label': raw_theory_labels,
         "main_curve": raw_theory_labels == main_curve_raw_theory_idx})
curves_df_raw_theory.sort_values(by=['r'], inplace=True)

px.line(curves_df_theory, x='theta', y='r', color='main_curve', line_group='label', title='Theoretical Curves').show()
px.line(curves_df_exp, x='theta', y='r', color='main_curve', line_group='label', title='Experimental Curves').show()
px.line(curves_df_raw_theory, x='theta', y='r', color='main_curve', line_group='label',
        title='Raw Theoretical Curves').show()

In [ ]:
main_curve_exp = curves_df_exp[curves_df_exp['main_curve'] == True]
main_curve_theory = curves_df_theory[curves_df_theory['main_curve'] == True]
main_curve_theory_raw = curves_df_raw_theory[curves_df_raw_theory['main_curve'] == True].groupby(
    by=['r']).mean().reset_index()

main_curve_theory['theta_s'] = scipy.signal.savgol_filter(main_curve_theory['theta'], 31, 3)
main_curve_exp['theta_s'] = scipy.signal.savgol_filter(main_curve_exp['theta'], 31, 3)
main_curve_theory_raw['theta_s'] = scipy.signal.savgol_filter(main_curve_theory_raw['theta'], 31, 3)

main_curve_theory['d_theta'] = main_curve_theory['theta_s'] - main_curve_theory['theta_s'].mean()
main_curve_exp['d_theta'] = main_curve_exp['theta_s'] - main_curve_exp['theta_s'].mean()
main_curve_theory_raw['d_theta'] = main_curve_theory_raw['theta_s'] - main_curve_theory_raw['theta_s'].mean()

main_curve_exp['E'] = main_curve_exp['r'] ** 2 / 2
main_curve_theory['E'] = main_curve_theory['r'] ** 2 / 2
main_curve_theory_raw['E'] = main_curve_theory_raw['r'] ** 2 / 2

main_curve_exp['d_E'] = main_curve_exp['E'] - main_curve_exp['E'].mean()
main_curve_theory['d_E'] = main_curve_theory['E'] - main_curve_theory['E'].mean()
main_curve_theory_raw['d_E'] = main_curve_theory_raw['E'] - main_curve_theory_raw['E'].mean()

main_curve_exp['dt/dE'] = np.gradient(main_curve_exp['theta_s'], main_curve_exp['E'])
main_curve_theory['dt/dE'] = np.gradient(main_curve_theory['theta_s'], main_curve_theory['E'])
main_curve_theory_raw['dt/dE'] = np.gradient(main_curve_theory_raw['theta_s'], main_curve_theory_raw['E'])

main_curve_exp['type'] = 'Experimental Data'
main_curve_theory['type'] = 'TDSE Calculations'
main_curve_theory_raw['type'] = 'Raw TDSE Calculations'

curves = pd.concat([main_curve_exp[(main_curve_exp["E"] > 0.015) & (main_curve_exp["E"] < 0.045)],
                    main_curve_theory[(main_curve_theory["E"] > 0.01) & (main_curve_theory["E"] < 0.032)]
                       ,
                    main_curve_theory_raw[(main_curve_theory_raw["E"] > 0.01) & (main_curve_theory_raw["E"] < 0.032)]])
curves["Group Delay"] = curves['dt/dE'] * 2
curves["Phase Delay"] = curves['theta_s'] / curves['E']

f1 = px.line(curves, x='d_E', y='theta', color='type', line_group='type', title='Curves')
f1.update_layout(yaxis_title='Relative Minimum Angle (rad)', xaxis_title='Relative Energy (a.u.)')
f2 = px.line(curves, x='d_E', y='Group Delay', color='type', line_group='type', title='Vortex Group Delay', height=800,
             width=1200)
f2.update_layout(yaxis_title='Group Delay (Atomic Units of Time)', xaxis_title='Relative Energy (a.u.)')
f2.add_hline(110, line_dash='dash', line_color='black', annotation=dict(text='110 a.u. (Laser Period)'))
f3 = px.line(curves, x='d_E', y='Phase Delay', color='type', line_group='type', title='Vortex Phase Delay', height=800,
             width=1200)
f3.update_layout(yaxis_title='Phase Delay (a.u)', xaxis_title='Relative Energy (a.u.)')
f1.show()
f2.show()
f3.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1, projection='polar')
plt.contourf(tt_resample, rr_resample_theory, signal_th, levels=100)
plt.contour(tt_us_th, rr_us_th, signal_th_dog, levels=5, colors='black')
plt.plot(main_curve_theory['theta_s'], main_curve_theory['r'], 'r')
plt.yticks([0.05, 0.15, 0.25])
plt.ylim(0, rr_resample_theory.max() * 1.1)

plt.subplot(1, 2, 2, projection='polar')
plt.contourf(tt_resample, rr_resample_exp, signal_exp, levels=100)
plt.contour(tt_us_exp, rr_us_exp, signal_exp_dog, levels=5, colors='black')
plt.plot(main_curve_exp['theta_s'], main_curve_exp['r'], 'r')
plt.yticks([0.1, 0.2, 0.3])
plt.ylim(0, rr_resample_exp.max() * 1.1)

In [ ]:
def find_phase_delay_curve(data_hist, low_sigma, r, plot=False):
    data_hist_stacked = np.vstack((data_hist, data_hist, data_hist))
    if plot:
        px.imshow(data_hist_stacked.T, origin='lower', aspect='auto').show()

    dog_filtered = skimage.filters.difference_of_gaussians(data_hist_stacked, low_sigma=low_sigma,
                                                           high_sigma=low_sigma * 2)[len(data_hist):2 * len(data_hist)]
    # dog_filtered_shift = np.roll(dog_filtered, int(len(data_hist)/2))
    # dog_filtered+=dog_filtered_shift

    if plot:
        f1 = px.imshow(dog_filtered.T, y=r, x=np.linspace(-np.pi, np.pi, len(data_hist)), origin='lower', aspect='auto')

    upsample_factor = 16
    rr_base, tt_base = np.meshgrid(r, np.linspace(-np.pi, np.pi, len(r)))
    print(rr_base.shape, tt_base.shape, dog_filtered.shape)
    rr, tt = np.meshgrid(r, np.linspace(-np.pi, np.pi, upsample_factor * len(r)))
    dog_filtered = scipy.interpolate.griddata(
            (rr_base.flatten(), tt_base.flatten()),
            dog_filtered.flatten(),
            (rr, tt),
            method='cubic'
    )
    rvec = rr[0]
    tvec = tt[:, 0]

    rel_min = scipy.signal.argrelmin(dog_filtered, axis=0)
    t_mc = tvec[rel_min[0]]
    r_mc = rvec[rel_min[1]]

    dbscan = DBSCAN(eps=0.03, min_samples=2)
    labels = dbscan.fit(np.array([t_mc, r_mc]).T).labels_
    main_curve_idx = np.argmax(np.bincount(labels[labels != -1]))
    main_curve = pd.DataFrame({'theta': t_mc, 'r': r_mc, 'label': labels, "main_curve": labels == main_curve_idx})
    main_curve['theta_s'] = scipy.signal.savgol_filter(main_curve['theta'], 21, 3)
    main_curve['E'] = main_curve['r'] ** 2 / 2
    main_curve['dt/dE'] = np.gradient(main_curve['theta_s'], main_curve['E'])
    main_curve['phase_diff'] = main_curve['dt/dE'] * 2
    main_curve['sigma'] = low_sigma
    main_curve = main_curve[main_curve['main_curve'] == True].sort_values(by=['r'])

    if plot:
        f1.add_trace(px.scatter(x=tvec[rel_min[0]], y=rvec[rel_min[1]]).data[0])
        f1.add_trace(px.line(x=main_curve['theta_s'], y=main_curve['r'], color='red'
                             ).data[0])
        f1.show()

    return main_curve


low_sigmas = np.linspace(15, 35, 10)

find_phase_delay_curve(signal_exp, 26.1, r_resample_exp, plot=True)
# curves=[find_phase_delay_curve(signal_exp, low_sigma, r_resample_exp) for low_sigma in low_sigmas]

In [ ]:
curves_df = pd.concat(curves)
curves_df.sort_values(by=['sigma', 'r'], inplace=True)
mask = (curves_df['E'] > 0.022) & (curves_df['E'] < 0.044)
px.line(curves_df[mask], x='E', y='phase_diff', color='sigma', line_group='sigma', title='Vortex Phase Delay').show()
px.density_heatmap(
        curves_df[mask],
        x='E', y='sigma', z='phase_diff',
        title='Vortex Phase Delay vs. Energy and Low Sigma',
        nbinsx=len(curves_df[mask]['E'].unique()), nbinsy=len(curves_df[mask]['sigma'].unique()),
        histfunc='avg'
).show()